In [1]:
!pip install langchain langchain_core langchain_community langchain-openai openai langchain_chroma tiktoken chromadb

In [2]:
from typing import List, Dict, Optional
from pydantic import BaseModel, Field, conlist

class InputData(BaseModel):
    topic: str = Field(..., description="The main topic")
    file_url: str = Field(..., description="URL of the file to be processed")
    file_type: str = Field(..., description="Type of the file (e.g., csv, json, xls, xlsx, xml)")
    lang: str = Field(..., description="Language of the document")

class Topic(BaseModel):
    topic_id: str = Field(..., description="Unique identifier for the topic")
    topic_name: str = Field(..., description="Human-readable name of the topic")
    keywords: List[str] = Field(..., description="List of keywords that define the topic")
    importance_score: float = Field(..., description="Score representing the importance of this topic in the clustering")
    description: Optional[str] = Field(None, description="Optional description of the topic")


class ClusterMetadata(BaseModel):
    num_clusters: int = Field(..., description="Total number of clusters generated")
    method: str = Field(..., description="Method used for clustering (e.g., K-means, LDA, etc.)")
    timestamp: Optional[str] = Field(None, description="Timestamp when the clustering process was completed")


class TopicCluster(BaseModel):
    cluster_id: str = Field(..., description="Unique identifier for the topic cluster")
    topics: List[Topic] = Field(..., description="List of topics within this cluster")
    central_topic: Optional[str] = Field(None, description="The central topic or most important topic within the cluster, if applicable")


class TopicClusteringOutput(BaseModel):
    clustering_algorithm: str = Field(..., description="Algorithm used to generate the clusters")
    clusters: conlist(TopicCluster) = Field(..., description="List of all topic clusters")
    metadata: ClusterMetadata = Field(..., description="Metadata regarding the clustering process")


In [3]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [4]:
!pip install langchain_google_genai

In [5]:
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [6]:
from enum import Enum

class FileType(Enum):
    PDF = 'pdf'
    CSV = 'csv'
    TXT = 'txt'
    MD = 'md'
    URL = "url"
    PPTX = 'pptx'
    DOCX = "docx"
    XLS = "xls"
    XLSX = "xlsx"
    XML = 'xml'

    GDOC = 'gdoc'
    GSHEET = "gsheet"
    GSLIDE = "gslide"
    GPDF = 'gpdf'

    YOUTUBE_URL = 'youtube_url'
    IMG = 'img'

In [7]:
class FileHandlerError(Exception):
    """Raised when a file content cannot be loaded. Used for tools which require file handling."""
    def __init__(self, message, url=None):
        self.message = message
        self.url = url
        super().__init__(self.message)

    def __str__(self):
        return f"{self.message}"

class ImageHandlerError(Exception):
    """Raised when an image cannot be loaded. Used for tools which require image handling."""
    def __init__(self, message, url):
        self.message = message
        self.url = url
        super().__init__(self.message)

    def __str__(self):
        return f"{self.message}"

class VideoTranscriptError(Exception):
    """Raised when a video transcript cannot be loaded. Used for tools which require video transcripts."""
    def __init__(self, message, url):
        self.message = message
        self.url = url
        super().__init__(self.message)

    def __str__(self):
        return f"{self.message}"

In [8]:
%%writefile document_loaders.py
from langchain_community.document_loaders import YoutubeLoader, PyPDFLoader, TextLoader, UnstructuredURLLoader, UnstructuredPowerPointLoader, Docx2txtLoader, UnstructuredExcelLoader, UnstructuredXMLLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
import os
import tempfile
import uuid
import requests
import gdown
from enum import Enum

class FileType(Enum):
    PDF = 'pdf'
    CSV = 'csv'
    TXT = 'txt'
    MD = 'md'
    URL = 'url'
    PPTX = 'pptx'
    DOCX = 'docx'
    XLS = 'xls'
    XLSX = 'xlsx'
    XML = 'xml'
    GDOC = 'gdoc'
    GSHEET = 'gsheet'
    GSLIDE = 'gslide'
    GPDF = 'gpdf'
    YOUTUBE_URL = 'youtube_url'
    IMG = 'img'

STRUCTURED_TABULAR_FILE_EXTENSIONS = {"csv", "xls", "xlsx", "gsheet", "xml"}

splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
)

def read_text_file(file_path):
    # Get the directory containing the script file
    script_dir = os.path.dirname(os.path.abspath(__file__))

    # Combine the script directory with the relative file path
    absolute_file_path = os.path.join(script_dir, file_path)

    with open(absolute_file_path, 'r') as file:
        return file.read()

def get_docs(file_url: str, file_type: str, verbose=True):
    file_type = file_type.lower()
    try:
        file_loader = file_loader_map[FileType(file_type)]
        docs = file_loader(file_url, verbose)

        return docs

    except Exception as e:
        print(e)
        print(f"Unsupported file type: {file_type}")
        raise FileHandlerError(f"Unsupported file type", file_url) from e

class FileHandler:
    def __init__(self, file_loader, file_extension):
        self.file_loader = file_loader
        self.file_extension = file_extension

    def load(self, url):
        # Generate a unique filename with a UUID prefix
        unique_filename = f"{uuid.uuid4()}.{self.file_extension}"

        # Download the file from the URL and save it to a temporary file
        response = requests.get(url)
        response.raise_for_status()  # Ensure the request was successful

        with tempfile.NamedTemporaryFile(delete=False, prefix=unique_filename) as temp_file:
            temp_file.write(response.content)
            temp_file_path = temp_file.name

        # Use the file_loader to load the documents
        try:
            loader = self.file_loader(file_path=temp_file_path)
        except Exception as e:
            print(f"No such file found at {temp_file_path}")
            raise FileHandlerError(f"No file found", temp_file_path) from e

        try:
            documents = loader.load()
        except Exception as e:
            print(f"File content might be private or unavailable or the URL is incorrect.")
            raise FileHandlerError(f"No file content available", temp_file_path) from e

        # Remove the temporary file
        os.remove(temp_file_path)

        return documents

def load_pdf_documents(pdf_url: str, verbose=False):
    pdf_loader = FileHandler(PyPDFLoader, "pdf")
    docs = pdf_loader.load(pdf_url)

    if docs:
        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found PDF file")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_csv_documents(csv_url: str, verbose=False):
    csv_loader = FileHandler(CSVLoader, "csv")
    docs = csv_loader.load(csv_url)

    if docs:
        if verbose:
            print(f"Found CSV file")
            print(f"Splitting documents into {len(docs)} chunks")

        return docs

def load_txt_documents(notes_url: str, verbose=False):
    notes_loader = FileHandler(TextLoader, "txt")
    docs = notes_loader.load(notes_url)

    if docs:

        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found TXT file")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_md_documents(notes_url: str, verbose=False):
    notes_loader = FileHandler(TextLoader, "md")
    docs = notes_loader.load(notes_url)

    if docs:

        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found MD file")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_url_documents(url: str, verbose=False):
    url_loader = UnstructuredURLLoader(urls=[url])
    docs = url_loader.load()

    if docs:
        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found URL")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_pptx_documents(pptx_url: str, verbose=False):
    pptx_handler = FileHandler(UnstructuredPowerPointLoader, 'pptx')

    docs = pptx_handler.load(pptx_url)
    if docs:

        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found PPTX file")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_docx_documents(docx_url: str, verbose=False):
    docx_handler = FileHandler(Docx2txtLoader, 'docx')
    docs = docx_handler.load(docx_url)
    if docs:

        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found DOCX file")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_xls_documents(xls_url: str, verbose=False):
    xls_handler = FileHandler(UnstructuredExcelLoader, 'xls')
    docs = xls_handler.load(xls_url)
    if docs:

        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found XLS file")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_xlsx_documents(xlsx_url: str, verbose=False):
    xlsx_handler = FileHandler(UnstructuredExcelLoader, 'xlsx')
    docs = xlsx_handler.load(xlsx_url)
    if docs:

        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found XLSX file")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_xml_documents(xml_url: str, verbose=False):
    xml_handler = FileHandler(UnstructuredXMLLoader, 'xml')
    docs = xml_handler.load(xml_url)
    if docs:

        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found XML file")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

class FileHandlerForGoogleDrive:
    def __init__(self, file_loader, file_extension='docx'):
        self.file_loader = file_loader
        self.file_extension = file_extension

    def load(self, url):

        unique_filename = f"{uuid.uuid4()}.{self.file_extension}"

        try:
            gdown.download(url=url, output=unique_filename, fuzzy=True)
        except Exception as e:
            print(f"File content might be private or unavailable or the URL is incorrect.")
            raise FileHandlerError(f"No file content available") from e

        try:
            loader = self.file_loader(file_path=unique_filename)
        except Exception as e:
            print(f"No such file found at {unique_filename}")
            raise FileHandlerError(f"No file found", unique_filename) from e

        try:
            documents = loader.load()
        except Exception as e:
            print(f"File content might be private or unavailable or the URL is incorrect.")
            raise FileHandlerError(f"No file content available") from e

        os.remove(unique_filename)

        return documents

def load_gdocs_documents(drive_folder_url: str, verbose=False):

    gdocs_loader = FileHandlerForGoogleDrive(Docx2txtLoader)

    docs = gdocs_loader.load(drive_folder_url)

    if docs:

        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found Google Docs files")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_gsheets_documents(drive_folder_url: str, verbose=False):
    gsheets_loader = FileHandlerForGoogleDrive(UnstructuredExcelLoader, 'xlsx')
    docs = gsheets_loader.load(drive_folder_url)
    if docs:

        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found Google Sheets files")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_gslides_documents(drive_folder_url: str, verbose=False):
    gslides_loader = FileHandlerForGoogleDrive(UnstructuredPowerPointLoader, 'pptx')
    docs = gslides_loader.load(drive_folder_url)
    if docs:

        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found Google Slides files")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_gpdf_documents(drive_folder_url: str, verbose=False):

    gpdf_loader = FileHandlerForGoogleDrive(PyPDFLoader,'pdf')

    docs = gpdf_loader.load(drive_folder_url)
    if docs:

        if verbose:
            print(f"Found Google PDF files")
            print(f"Splitting documents into {len(docs)} chunks")

        return docs

def load_docs_youtube_url(youtube_url: str, verbose=True) -> str:
    try:
        loader = YoutubeLoader.from_youtube_url(youtube_url, add_video_info=True)
    except Exception as e:
        print(f"No such video found at {youtube_url}")
        raise VideoTranscriptError(f"No video found", youtube_url) from e

    try:
        docs = loader.load()
        length = docs[0].metadata["length"]
        title = docs[0].metadata["title"]

    except Exception as e:
        print(f"Video transcript might be private or unavailable in 'en' or the URL is incorrect.")
        raise VideoTranscriptError(f"No video transcripts available", youtube_url) from e

    if verbose:
        print(f"Found video with title: {title} and length: {length}")
        print(f"Combined documents into a single string.")
        print(f"Beginning to process transcript...")

    split_docs = splitter.split_documents(docs)

    return split_docs

llm_for_img = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

def generate_docs_from_img(img_url, verbose: bool=False):
    message = HumanMessage(
    content=[
            {
                "type": "text",
                "text": "Give me a summary of what you see in the image. It must be 3 detailed paragraphs.",
            },
            {"type": "image_url", "image_url": img_url},
        ]
    )

    try:
        response = llm_for_img.invoke([message]).content
        print(f"Generated summary: {response}")
        docs = Document(page_content=response, metadata={"source": img_url})
        split_docs = splitter.split_documents([docs])
    except Exception as e:
        print(f"Error processing the request due to Invalid Content or Invalid Image URL")
        raise ImageHandlerError(f"Error processing the request", img_url) from e

    return split_docs

file_loader_map = {
    FileType.PDF: load_pdf_documents,
    FileType.CSV: load_csv_documents,
    FileType.TXT: load_txt_documents,
    FileType.MD: load_md_documents,
    FileType.URL: load_url_documents,
    FileType.PPTX: load_pptx_documents,
    FileType.DOCX: load_docx_documents,
    FileType.XLS: load_xls_documents,
    FileType.XLSX: load_xlsx_documents,
    FileType.XML: load_xml_documents,
    FileType.GDOC: load_gdocs_documents,
    FileType.GSHEET: load_gsheets_documents,
    FileType.GSLIDE: load_gslides_documents,
    FileType.GPDF: load_gpdf_documents,
    FileType.YOUTUBE_URL: load_docs_youtube_url,
    FileType.IMG: generate_docs_from_img
}

Overwriting document_loaders.py


In [9]:
!pip install pypdf fpdf youtube-transcript-api pytube unstructured python-pptx docx2txt networkx pandas xlrd openpyxl gdown pytest PyPDF2 psutil

In [10]:
!pip uninstall -y nltk
!pip install nltk
import nltk
nltk.download('punkt')

Found existing installation: nltk 3.9.1
Uninstalling nltk-3.9.1:
  Successfully uninstalled nltk-3.9.1
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
def read_text_file(file_path):
    # Get the current working directory
    script_dir = os.getcwd()

    # Combine the script directory with the relative file path
    absolute_file_path = os.path.join(script_dir, file_path)

    try:
        with open(absolute_file_path, 'r') as file:
            content = file.read()
        return content
    except FileNotFoundError:
        # Handle the case where the file is not found
        print(f"File not found: {absolute_file_path}")
        return None

In [12]:
from langchain_openai.embeddings import OpenAIEmbeddings
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [13]:
from langchain_openai import ChatOpenAI

llm_chat_openai = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [14]:
from langchain_google_genai import GoogleGenerativeAI

llm_google_genai = GoogleGenerativeAI(model="gemini-1.5-pro")

In [15]:
from langchain.prompts import PromptTemplate

hyde_prompt_template = """
Generate a hypothetical answer to the user's query.
Query: {query}
Hypothetical Answer:
"""

hyde_prompt = PromptTemplate(input_variables=["query"], template=hyde_prompt_template)

hyde_chain = hyde_prompt | llm_google_genai

In [16]:
from langchain.schema import Document
from langchain.chains import  HypotheticalDocumentEmbedder
from langchain.vectorstores import Chroma

def build_docsearch(documents: List[Document], embedding_model):
  embeddings = HypotheticalDocumentEmbedder(
    llm_chain=hyde_chain,
    base_embeddings=embedding_model
  )

  docsearch = Chroma.from_documents(documents, embeddings)

  return docsearch

In [17]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser(pydantic_object=TopicClusteringOutput)
format_instructions = parser.get_format_instructions()

In [18]:
prompt_template = """
Analyze the following document and perform topic clustering. Based on the document's content, select the most appropriate clustering algorithm, identify relevant topics, group them into clusters, and determine the importance of each topic within its cluster.

Main Topic:
-----------------------------
{topic}

Context:
-----------------------------
{context}

Formatting:
-----------------------------
{format_instructions}

Ensure the following:
- Automatically select the clustering algorithm based on the document.
- Provide metadata about the number of clusters and the method used.
- Identify and cluster topics, including their importance score and relevant keywords.
- Highlight any central topic within each cluster if applicable.
"""

In [19]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["topic", "context"],
    partial_variables={"format_instructions": format_instructions}
)

In [20]:
chain = prompt | llm_chat_openai | parser

In [21]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

In [22]:
from document_loaders import get_docs

def run_chain(input_data: InputData):
    documents = get_docs(input_data.file_url, input_data.file_type, verbose=True)
    if not documents:
        print("No documents loaded.")
        return

    split_docs = splitter.split_documents(documents)

    docstore = build_docsearch(split_docs, embedding_model)

    user_query = "Develop a topic clustering based in this main topic: "+input_data.topic

    relevant_docs = docstore.similarity_search(user_query)

    context = "\n".join([doc.page_content for doc in relevant_docs])

    output = chain.invoke({'topic': input_data.topic, 'context': context})

    print(output)

    del docstore

    return output

In [23]:
if __name__ == "__main__":
    input_data = InputData(
        topic="LLM",
        file_url="https://raw.github.com/AaronSosaRamos/mission-flights/main/files-for-test/text.pdf",
        file_type="pdf",
        lang="en"
    )

    result = run_chain(input_data)

Found PDF file
Splitting documents into 2 chunks


{'clustering_algorithm': 'K-means', 'clusters': [{'cluster_id': 'cluster_1', 'central_topic': 'Applications of LLMs', 'topics': [{'topic_id': 'topic_1', 'topic_name': 'Healthcare Applications', 'keywords': ['healthcare', 'medical documentation', 'research'], 'importance_score': 0.9, 'description': 'Use of LLMs in healthcare for documentation and research.'}, {'topic_id': 'topic_2', 'topic_name': 'Education Applications', 'keywords': ['education', 'intelligent tutoring systems'], 'importance_score': 0.8, 'description': 'Implementation of LLMs in educational tools and systems.'}, {'topic_id': 'topic_3', 'topic_name': 'Business Applications', 'keywords': ['business', 'customer service', 'chatbots', 'virtual assistants'], 'importance_score': 0.85, 'description': 'Leveraging LLMs for improving customer service in businesses.'}]}, {'cluster_id': 'cluster_2', 'central_topic': 'Challenges of LLMs', 'topics': [{'topic_id': 'topic_4', 'topic_name': 'Bias in LLMs', 'keywords': ['bias', 'societal 

In [24]:
result

{'clustering_algorithm': 'K-means',
 'clusters': [{'cluster_id': 'cluster_1',
   'central_topic': 'Applications of LLMs',
   'topics': [{'topic_id': 'topic_1',
     'topic_name': 'Healthcare Applications',
     'keywords': ['healthcare', 'medical documentation', 'research'],
     'importance_score': 0.9,
     'description': 'Use of LLMs in healthcare for documentation and research.'},
    {'topic_id': 'topic_2',
     'topic_name': 'Education Applications',
     'keywords': ['education', 'intelligent tutoring systems'],
     'importance_score': 0.8,
     'description': 'Implementation of LLMs in educational tools and systems.'},
    {'topic_id': 'topic_3',
     'topic_name': 'Business Applications',
     'keywords': ['business',
      'customer service',
      'chatbots',
      'virtual assistants'],
     'importance_score': 0.85,
     'description': 'Leveraging LLMs for improving customer service in businesses.'}]},
  {'cluster_id': 'cluster_2',
   'central_topic': 'Challenges of LLMs'